In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time

## Extract COP archive side events data


In [2]:
url = 'https://seors.unfccc.int/applications/seors/reports/archive.html'
r = requests.get(url).content.decode()
soup = bs(r, 'html')

In [9]:
# Compile the historical events
items = soup.select('[name=session_id] option[value]')
event_table = pd.DataFrame(data={"code":np.nan,"event":np.nan},index=np.arange(len(items))[1:])
for i in np.arange(len(items))[1:]:
    this_item = str(items[i])
    event_table.loc[i,"code"] = this_item.split("=")[1].split(">",1)[0][1:-1]
    event_table.loc[i,"event"] = this_item.split(">")[1][:-8]
event_table_cop = event_table[event_table["code"].str.contains("COP")]
event_table_cop = event_table_cop[event_table_cop["code"] != "COP 23"]
event_table_cop = event_table_cop.set_index("code").reset_index()
for i in event_table_cop.index:
    event_table_cop.loc[i,"name"] = "COP"+ str(28-i).zfill(2)
event_table_cop.set_index("code").to_csv("COP_archive_side_events_2024.csv")
event_table_cop

,code,event,name
0,COP 28,UN Climate Change Conference November/December...,COP28
1,COP 27,UN Climate Change Conference November 2022 (CO...,COP27
2,COP26,UN Climate Change Conference November 2021 (CO...,COP26
3,COP 25,UN Climate Change Conference December 2019 (CO...,COP25
4,COP 24,UN Climate Change Conference December 2018 (CO...,COP24
5,COP23,UN Climate Change Conference November 2017 (CO...,COP23
6,COP22,UN Climate Change Conference November 2016 (CO...,COP22
7,COP21,UN Climate Change Conference December 2015 (CO...,COP21
8,COP20,COP 20/CMP 10,COP20
9,COP19,COP 19/ CMP 9,COP19


### Extract data 

In [4]:
# set up chrome driver
# see start of the file on how to set this up 
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless') # make it work without opening a window 
driver = webdriver.Chrome("C:\Program Files\Java\chromedriver-win64\chromedriver.exe", options=options) # USER DEFINED FILEPATH

In [8]:
for i in event_table_cop.index:
    event_code = event_table_cop.loc[i,"code"]
    event_name = event_table_cop.loc[i,"name"]

    # select the category at the drop down 
    driver.get(url)
    select = Select(driver.find_element_by_name('session_id')) # select category by the drop down 
    select.select_by_value(event_code) # select by value
    time.sleep(1)
    driver.find_element_by_name('Submit').submit() # click the "refresh" button 
    time.sleep(1)

    page_source = driver.page_source
    soup_events = bs(page_source, 'html')

    table_events = soup_events.find_all('table')[4]
    df_events = pd.read_html(str(table_events))[0]
    df_events = df_events[df_events["Date"].isna()==False]
    df_events = df_events[df_events["Organizer"].isna()==False] # some attachments are shifted weird
    df_events = df_events.reset_index()
    df_events = df_events.drop(columns={"Attachments","index"})
    df_events = df_events.drop_duplicates()
    df_events = df_events.drop
    df_events.index.names = ['Code']
    df_events = df_events.reset_index()
    df_events["SE_Code"] = event_name+ "-" + df_events["Code"].astype(str).str.zfill(3)
    
    if i == event_table_cop.index[0]:
        df_events_all = df_events
    else:
        df_events_all = pd.concat([df_events_all,df_events])
        
df_events_all.set_index("SE_Code").to_csv("COP_archive_side_events_compiled_raw_2024.csv")
df_events_all

,Code,Title/theme/speakers,Organizer,Date,SE_Code
0,0,COP 28: Facing the challenges for developing c...,Yvonne Miller Berlie Third World Network (TWN...,"Fri, 01 Dec 2023",COP28-000
1,1,The Global Stocktake: How to strengthen climat...,Amir Khouzam International Committee of the R...,"Fri, 01 Dec 2023",COP28-001
2,2,Change through participation: Open Dialogues a...,Zuzanna Borowska Polish Ecological Club (PEC)...,"Fri, 01 Dec 2023",COP28-002
3,3,Partnerships for Sustainable Finance: Opportun...,Samuel Confidence Dotse HATOF Foundation samu...,"Fri, 01 Dec 2023",COP28-003
4,4,Proven climate solutions: following IP&LC lead...,Ashley Emerson Health In Harmony (HIH) ashley...,"Fri, 01 Dec 2023",COP28-004
...,...,...,...,...,...
109,109,Russian Business Forum: Corporations and Regio...,Evgeniy Sokolov National Carbon Sequestration...,"Thu, 11 Dec 2003",COP09-109
110,110,Standards to ensure high quality LULUCF projec...,Olivia Tanujaya Yayasan Pelangi (PELANGI) oli...,"Thu, 11 Dec 2003",COP09-110
111,111,Linking Article 2 and Article 6: Experiments i...,Jean-Pascal van Ypersele Belgium vanypersele@...,"Fri, 12 Dec 2003",COP09-111
112,112,South-North Dialogue - Equity in the Greenhous...,"Bernd Brouns Wuppertal Institute for Climate,...","Fri, 12 Dec 2003",COP09-112


In [10]:
df_events

,Code,Title/theme/speakers,Organizer,Date,SE_Code
0,0,Commitment of emissions to human development: ...,Jiahua Pan Research Centre for Sustainable De...,"Mon, 01 Dec 2003",COP09-000
1,1,Development and transfer of technologies Prese...,Florin Vladu Secretariat of the United Nation...,"Mon, 01 Dec 2003",COP09-001
2,2,The Great Warming Keanu Reeves and Alanis Mori...,Colette Cardinal Canada colette.cardinal@dfai...,"Mon, 01 Dec 2003",COP09-002
3,3,The interaction between the EU emissions tradi...,Jos Sijm Energy Research Centre of the Nether...,"Mon, 01 Dec 2003",COP09-003
4,4,Collaborative Partnership on Forests: an inter...,Mia Soderlund United Nations (UN) soderlund@u...,"Mon, 01 Dec 2003",COP09-004
...,...,...,...,...,...
109,109,Russian Business Forum: Corporations and Regio...,Evgeniy Sokolov National Carbon Sequestration...,"Thu, 11 Dec 2003",COP09-109
110,110,Standards to ensure high quality LULUCF projec...,Olivia Tanujaya Yayasan Pelangi (PELANGI) oli...,"Thu, 11 Dec 2003",COP09-110
111,111,Linking Article 2 and Article 6: Experiments i...,Jean-Pascal van Ypersele Belgium vanypersele@...,"Fri, 12 Dec 2003",COP09-111
112,112,South-North Dialogue - Equity in the Greenhous...,"Bernd Brouns Wuppertal Institute for Climate,...","Fri, 12 Dec 2003",COP09-112
